## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2023-09-28 18:18:07--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:601f:18::a27d:912
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2023-09-28 18:18:08--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc00b13ed3ce82318bbe8d8abf70.dl.dropboxusercontent.com/cd/0/get/CEmUrhCjbaxMf32EhLCOUwCNP1t_PlIHTes4clNrlCr5wvxDd-fTjAbyxJBxFGVNyeJnRPna765dlSjmnzPyfttqAh8w1T5yBB6ClwIujQ1p5SmwYFmg1c75STYVr-Ssb08/file?dl=1# [following]
--2023-09-28 18:18:08--  https://uc00b13ed3ce82318bbe8d8abf70.dl.dropboxusercontent.com/cd/0/get/CEmUrhCjbaxMf32EhLCOUwCNP1t_PlIHTes4clNrlCr5wvxDd-fTjAbyxJBxFGVNyeJnRPna765dlSjmnzPyfttqAh8w1T5yBB6ClwIujQ1p5SmwYFmg1c75STYVr-Ssb0

In [2]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [4]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(w.lower()) for w in data]

In [5]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [6]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings.

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [7]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [8]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.6939209 , -0.12102875,  2.327667  ,  1.971942  ,  0.8204293 ,
        1.4694661 ,  3.0584528 , -3.2430906 ,  0.56943727,  1.9937505 ,
       -1.3567373 , -0.12684797,  4.7296863 ,  0.5064142 ,  3.5926297 ,
        0.11764541, -0.7350521 , -1.7153277 , -0.2685522 , -3.5200112 ,
       -2.6072717 , -0.74033445, -2.7041762 ,  0.09582313,  0.08450498,
       -1.4486384 , -0.69195396,  0.08514941,  2.8444333 ,  0.84735554,
       -0.13701604,  0.1881997 ], dtype=float32)

In [9]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9513570070266724),
 ('cheese', 0.9303231835365295),
 ('chocolate', 0.9286747574806213),
 ('sauce', 0.9226927757263184),
 ('corn', 0.9217368364334106),
 ('pasta', 0.9211599230766296),
 ('fruit', 0.9161158204078674),
 ('soup', 0.9145574569702148),
 ('vodka', 0.9141542315483093),
 ('butter', 0.911596953868866)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts.

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [11]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.4922019839286804),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [12]:
words = model.index_to_key[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [13]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(w) for w in words])

In [14]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [15]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components=2).fit_transform(word_vectors)

word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(axis=0)) / word_vectors_pca.std(axis=0)
# and maybe MORE OF YOUR CODE here :)

In [16]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [17]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [18]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [19]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(2).fit_transform(word_vectors)

In [20]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1049', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [21]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    vector = np.zeros([model.vector_size], dtype='float32')
    # изначально пустой вектор по длине эмбеддинга в модели
    cnt = 0
    for w in tokenizer.tokenize(phrase.lower()):
      if w in model.key_to_index:
        vector += model.get_vector(w) # каждый раз добавляем эмбеддинг в вектор, то есть vector[0] += embedding(w)[0]
        cnt += 1 # считаем сколько раз встретились слова
    vector /= cnt if cnt > 0 else 1 # mean
    return vector



In [22]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [23]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(d) for d in chosen_phrases])

In [24]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [25]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [26]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1094', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [27]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [37]:
from bokeh.core.enums import SizingModeType
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    q_vector = get_phrase_embedding(query)
    similarity = []
    for d in data_vectors:
      n1, n2 = np.linalg.norm(q_vector), np.linalg.norm(d)
      if n1 != 0 and n2 != 0:
        similarity.append(q_vector @ d / n1 / n2)
      else:
        similarity.append(q_vector @ d / (1e4 + n1) / (1e4 + n2))

    top_k_indices = np.argpartition(similarity, -k)[-k:]

    sorted_top_k = top_k_indices[np.argsort(-np.array(similarity)[top_k_indices])]

    return [data[i] for i in sorted_top_k]

In [38]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [39]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [40]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

In [41]:
!pip install nearpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 13.4 MB/s eta 0:00:00


In [42]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections

def find_nearest_nearpy(query, k=10):
    q_vector = get_phrase_embedding(query)
    dimension = q_vector.shape[0]

    rbp = RandomBinaryProjections('rbp', 10)

    engine = Engine(dimension, lshashes=[rbp])

    for i, d in enumerate(data_vectors):
        engine.store_vector(d, i)

    N = engine.neighbours(q_vector)

    top_k_neighbors = N[:k]
    top_k_indexes = [int(item[1]) for item in top_k_neighbors]
    return [data[i] for i in top_k_indexes]


In [43]:
results = find_nearest_nearpy(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I win this?
What do I do to enter the line of event management?
What can I do for this world?
How do I start the dead ubislate 7cz tablet?
How do you use the components in Final Fantasy XIII?
How do I find the factor of the number?
How do I solve this limit as n approaches infinity? What trick do I need to use in this?
How do I attach an HDD to the Xbox 360? And how do I copy games from the PC to the Xbox?
How do I choose the subject for my blog?
How do I start to code?



AssertionError: ignored

In [44]:
from sklearn.neighbors import NearestNeighbors

nn_model = NearestNeighbors(metric='cosine').fit(data_vectors)

def find_nearest_sklearn(query, k=10):
    q_vector = get_phrase_embedding(query)
    q_vector = q_vector.reshape(1, -1)
    distances, indices = nn_model.kneighbors(q_vector, n_neighbors=k)
    nearest_indices = indices[0]
    sorted_nearest_indices = [index for _, index in sorted(zip(distances[0], nearest_indices))]
    return [data[i] for i in sorted_nearest_indices]


In [45]:
results = find_nearest_sklearn(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



Мне кажется реузльтаты с использованием sklearn точнее, хотя все это субъективно.

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.

## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully).

For our system we choose two kindred Slavic languages: Ukrainian and Russian.

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [ ]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for ukrainian and russian.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Datasets/week01-embeddings/cc.uk.300.vec.zip

cc.ru.300.vec.zip  cc.uk.300.vec.zip


In [ ]:
!unzip /content/drive/MyDrive/Datasets/week01-embeddings/cc.uk.300.vec.zip

Archive:  /content/drive/MyDrive/Datasets/week01-embeddings/cc.uk.300.vec.zip
  inflating: cc.uk.300.vec           


In [ ]:
!unzip /content/drive/MyDrive/Datasets/week01-embeddings/cc.ru.300.vec.zip

Archive:  /content/drive/MyDrive/Datasets/week01-embeddings/cc.ru.300.vec.zip
  inflating: cc.ru.300.vec           


In [ ]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [ ]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [ ]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

In [ ]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [ ]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.230317160487175),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587750673294),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872766315937042),
 ('конструктивен', 0.21796567738056183)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [ ]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
path = "/content/drive/MyDrive/Datasets/week01-embeddings/"

In [ ]:
uk_ru_train, X_train, Y_train = load_word_pairs(path + "ukr_rus.train.txt")

In [ ]:
uk_ru_test, X_test, Y_test = load_word_pairs(path + "ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [ ]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.8234528303146362),
 ('август', 0.812049925327301),
 ('декабрь', 0.803900420665741)]

In [ ]:
[i[0] for i in ru_emb.most_similar(august)]

['апрель',
 'июнь',
 'март',
 'сентябрь',
 'февраль',
 'октябрь',
 'ноябрь',
 'июль',
 'август',
 'декабрь']

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [ ]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        neighbours = [i[0] for i in ru_emb.most_similar(mapped_vectors[i])][:topn]
        if ru in neighbours:
          num_matches += 1

    precision_val = num_matches / len(pairs)
    return precision_val


In [ ]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [ ]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
print(precision_top5)
assert precision_top5 >= 0.813

0.813953488372093


## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal.
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [ ]:
def learn_transform(X_train, Y_train):
    """
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    matrix = X_train.T @ Y_train
    u, sigma, vt = np.linalg.svd(matrix)
    W_star = np.dot(u, vt)

    return W_star

In [ ]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237906694412231),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [ ]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [ ]:
with open(path + "fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [ ]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    emb = []
    for s in sentence.split():
      if s in uk_emb.key_to_index:
        emb.append(uk_emb[s])
      else:
        emb.append(uk_emb["*"])
    if len(emb) == 0:
        return sentence
    else:
      transform = np.matmul(emb, W)
      trslt = []
      for t in transform:
        trslt.append(ru_emb.most_similar(t)[0][0])
      return ' '.join([item if item != '*' else '*' * 4 for item in trslt])

Было принято решение не найденные слова заменять на ****. 4 потому что средняя длина слов во всех предложениях 4.

In [ ]:
np.mean([len(s) for sent in uk_sentences for s in sent.split() ])

4.13484251968504

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – ****

src: як була собі лисичка та зробила хатку, та й живе. а це приходять холоди. от лисичка замерзла та й побігла в село вогню добувать, щоб витопити. прибігає до одної баби та й каже:
dst: как была себе лисичка и сделала **** и и **** а оно приходят **** из лисичка замерзла и и побежала во село огня **** чтобы **** прибегает к одной бабы и и ****

src: — здорові були, бабусю! з неділею... позичте мені огню, я вам одслужу.
dst: — здоровые **** **** со **** **** мне **** мной тебе ****

src: — добре, — каже, — лисичко - сестричко. сідай погрійся трохи, поки я пиріжечки повибираю з печі!
dst: — **** — **** — **** – **** садись **** **** пока мной пирожки **** со ****

src: а баба макові пиріжки пекла. от баба вибирає пиріжки та на столі кладе, щоб прохололи; а лисичка підгляділа та за пиріг, та з хати... виїла мачок із середини, а туди напхала сміттячка, стулила та й біжить.
dst: а бабка маковые пирожки **** 

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -